In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list


In [ ]:
!kaggle datasets download -d dollyprajapati182/balanced-raf-db-dataset-7575-grayscale


In [ ]:
!unzip balanced-raf-db-dataset-7575-grayscale.zip -d /content/raf_db


In [ ]:
import os

data_path = '/content/raf_db'
print(os.listdir(data_path))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


In [ ]:
train_dir = os.path.join(data_path, 'train')
val_dir = os.path.join(data_path, 'val')

print("Train classes:", os.listdir(train_dir))
print("Validation classes:", os.listdir(val_dir))


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(75, 75),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(75, 75),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(75, 75, 1)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_accuracy',  
    patience=10,            
    verbose=1,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    '/content/raf_db_best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

history = model.fit(
    train_generator,
    epochs=100,  
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()


In [ ]:
val_generator.reset()
loss, accuracy = model.evaluate(val_generator)
print(f"Overall Validation Accuracy: {accuracy*100:.2f}%")


In [ ]:
val_generator.reset()
predictions = model.predict(val_generator)
pred_labels = np.argmax(predictions, axis=1)

true_labels = val_generator.classes
class_names = list(val_generator.class_indices.keys())

# Classification report
print("\nClassification Report:\n")
report = classification_report(true_labels, pred_labels, target_names=class_names, output_dict=True)
print(classification_report(true_labels, pred_labels, target_names=class_names))

# Print per-emotion accuracy (recall)
print("\nEmotion-wise Accuracy (Recall):\n")
for emotion, metrics in report.items():
    if emotion in ["accuracy", "macro avg", "weighted avg"]:
        continue
    print(f"{emotion.capitalize():<10} : {metrics['recall']*100:.2f}%")

# Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
model.save('/content/raf_db_model.h5')
print("Model saved successfully!")
